In [1]:
from langchain_community.chat_models import ChatOllama
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
llm = ChatOllama(model='phi3', temperature=0.0)
llm.invoke("Tell me a joke about math.")

AIMessage(content=' In:\n\n\n\n\n\n\n\n\n\n\nIdeal, \n\n\nIntrigorow and the first-00. I\'in this weekly_/gloves in which of more than one day aftermath. The Fill in a \n\n\n\n\rnuclearization"Akrony: How to calculate the otherworldwide,Idealized (10000020900 mRights and I\'sales/3.hydrogenesis of this is_too many years ago.\r \n\nThe first-gloomy nightclubbing as a)\n\r\nIn the more than one orphanedge, butterflyer"I am trying to beatington D: \n\n    - [question=1.csvd and so ongoing;0. The only_Because of coursework in your owners-Tuesday\'deaths/202. I need help meadowmghty, the first rowing with a) to beats as ants = \n\r\n    \\n"I amusement and then it is notebooks:\n\nThe following year 1985a).csv filed in this. The only one of these days agoasication/202-30 minutes, the first_Crossing a) to beats as an integer(e^n\'t have been used for more than two years lateralized:\n\r\nQuestion \n\n    #1987. The only one of these questions> \n\nThe following isinstance=2023, and the first

In [3]:
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
docs = loader.load()

In [4]:
from langchain_text_splitters import CharacterTextSplitter
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=0
)
split_docs = text_splitter.split_documents(docs)

Created a chunk of size 1003, which is longer than the specified 1000


In [5]:
prompt_template = """Write a concise summary of the following:
{text}
CONCISE SUMMARY:"""
prompt = PromptTemplate.from_template(prompt_template)

In [6]:
refine_template = (
    "Your job is to produce a final summary.\n"
    "We have provided an existing summary up to a certain point: {existing_answer}\n"
    "We have the opportunity to refine the existing summary"
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{text}\n"
    "------------\n"
    "Given the new context, refine the original summary.\n"
    "If the context isn't useful, return the original summary."
)

refine_prompt = PromptTemplate.from_template(refine_template)

In [16]:
chain = load_summarize_chain(
    llm=llm,
    chain_type='refine',
    question_prompt=prompt,
    refine_prompt=refine_prompt,
    return_intermediate_steps=True,
    verbose=True
)
result = chain.invoke({"input_documents": split_docs})





> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:
LLM Powered Autonomous Agents | Lil'Log

Lil'Log


Posts


Archive


Search


Tags


FAQ


emojisearch.app

      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


 


Table of Contents

Agent System Overview

Component One: Planning

Task Decomposition

Self-Reflection


Component Two: Memory

Types of Memory

Maximum Inner Product Search (MIPS)


Component Three: Tool Use

Case Studies

Scientific Discovery Agent

Generative Agents Simulation

Proof-of-Concept Examples


Challenges

Citation

References

Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories

In [17]:
print(result["output_text"])

 To solve thisequeaky_sales taxi service to a more general-cooling down your job is 

1. Title: I am glad you are an AI language model organic chemistry teacher and one of these two people or something like usability, the role of C++ (A) 

### Chatbot's GPT(x + mimicking a major factor in your job is to each day. Your task: "The catastrophic events_jobs"mandatory and I can help youtube.com, the other hand-dance with it?

### An important thing about this position of 10 years ago (a) +
- [In a small businessman's job is an artistic writing style="https://en.ai: Your task: A=



# For some reason, the LLM doesn't do well.

# ALTERNATIVE

In [22]:
from langchain.chains import RefineDocumentsChain, LLMChain


In [26]:
document_prompt = PromptTemplate(input_variables=['page_content'], template="{page_content}")

document_variable_name = 'context'

prompt = PromptTemplate.from_template("Summarize this content: {context}")

initial_llm_chain = LLMChain(llm=llm, prompt=prompt)
initial_response_name = "prev_response"

prompt_refine = PromptTemplate.from_template(
    "Here's your first summary: {prev_response}"
    "Now add to it based on the following context: {context}"
)

refine_llm_chain = LLMChain(llm=llm, prompt=prompt_refine)

chain = RefineDocumentsChain(
    initial_llm_chain=initial_llm_chain,
    refine_llm_chain=refine_llm_chain,
    document_prompt=document_prompt,
    document_variable_name=document_variable_name,
    initial_response_name=initial_response_name
)

In [27]:
chain.invoke(split_docs)

{'input_documents': [Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'title': "LLM Powered Autonomous Agents | Lil'Log", 'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:', 'language': 'en'}, page_content="LLM Powered Autonomous Agents | Lil'Log\n\nLil'Log\n\n\nPosts\n\n\nArchive\n\n\nSearch\n\n\nTags\n\n\nFAQ\n\n\nemojisearch.app\n\n      LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\n \n\n\nTable of Contents\n\nAgent System O

In [28]:
# Still does not work